In [1]:
#!/usr/bin/python
from __future__ import print_function
from future import standard_library
standard_library.install_aliases()
from builtins import input

import pytumblr
import yaml
import sys
import os
import code
import requests
import urllib.request
import re

from requests_oauthlib import OAuth1Session
from PIL import Image
from io import BytesIO

In [2]:
# Get token
yaml_path = os.path.expanduser('~') + '/.tumblr'
yaml_file = open(yaml_path, "r")
tokens = yaml.safe_load(yaml_file)
yaml_file.close()

In [3]:
# Use token to be able to use the client
client = pytumblr.TumblrRestClient(
    tokens['consumer_key'],
    tokens['consumer_secret'],
    tokens['oauth_token'],
    tokens['oauth_token_secret']
)

In [20]:
# Get the overall amount of likes
amount_likes = client.likes()["liked_count"]

In [21]:
def save(url,content_type,post_url):
    
    '''A saver funtion for downloading content based on URL'''
    
    post_url = post_url.replace("/",'|')
    if content_type == "video":
        try:
            urllib.request.urlretrieve(url, 'videos/'+post_url+'.mp4')
        except:
            print(post_url+" didn't work")
    else:
        img_data = requests.get(url).content
        with open('images/'+post_url+'.png', 'wb') as handler:
            handler.write(img_data)

            

In [33]:
# The batches have this specific pattern because I used this method halfway through downloading my stuff
batches = [(i,i+49) for i in range(1299,6364,50)]

In [58]:
for batch in batches:
    # Iterate over batches of size 49 to create as little requests as possible
    request = client.likes(offset=batch[0],limit=batch[1])
    for i in range(len(request["liked_posts"])):
        # Iterate over the liked posts of the batch
        post = request["liked_posts"][i]
        if len(post):
            content_type = post['type']
            # To check whether I forgot a to handle a contetn type 
            if ('photo' or 'text' or 'video') not in content_type:
                print(content_type)
            post_url = post["post_url"]
            if content_type == "photo":
                # If only one photo, download, otherwise iterate over them an download
                if len(post["photos"]) == 1:
                    url = post["photos"][0]["original_size"]['url']
                    save(url,content_type,post_url)
                else:
                    post_url += "_{}"
                    for j in range(len(post["photos"])):
                        url = post["photos"][j]["original_size"]['url']
                        save(url,content_type,post_url.format(j))
            elif content_type == "text":
                # Get the body as an HTML style string. Use Regex to extract photo URLs
                # If only one photo, download, otherwise iterate over them an download
                content = post["body"]
                url_s = re.findall(r'src="(http[s]:[\S]*media\.tumblr\.com[\S]*)"',content)
                if len(url_s) == 1:
                    save(url_s[0],content_type,post_url)
                else:
                    post_url += "_{}"
                    for j in range(len(url_s)):
                        save(url_s[j],content_type,post_url.format(j))
            elif content_type == "video":
                # Download the video file
                url_s = post["video_url"]
                save(url_s,content_type,post_url)





        sys.stdout.write("\r" + str(batch[0]+i) + '/'+ str(amount_likes))
        sys.stdout.flush()

1606/7663

KeyboardInterrupt: 

In [17]:
# Test how timestamp work for the likes() function. Only returns 50 posts at a time
like = client.likes(before=1541317099,after=1388561878,limit=10000)

# To do:
- Solve the problem that I only can retrieve the latest 1000 likes
- Use threading for writing on disc and doing requests
- Try deeplearning for further sorting than just for NSFW content

# Nudity detection
### To divide NSFW content from the rest

In [ ]:
images = os.listdir("images/")

from nudity import Nudity
import shutil
from PIL import Image

nudity = Nudity();

for i,image in enumerate(images):
    # Before checking for NSFW content, see whether the file is a GIF.\
    try:
        gif = Image.open('images/' + image)
        try:
            gif.seek(1)
        except EOFError:
            isanimated = False
        else:
            shutil.move('images/' + image, 'gif/' + image)
            isanimated = True
        if (not isanimated and nudity.score('images/' + image) > 0.2 ):
            shutil.move('images/' + image, 'images/nude/' + image)
    except OSError:
        print(image)
    
    sys.stdout.write("\r" + str(i) + '/'+ str(len(images)))
    sys.stdout.flush()

    # For boolean return 
    # print(nudity.has())



In [76]:
# NOT USED YET
# 1000 request per month for free deepai
import requests
r = requests.post(
    "https://api.deepai.org/api/nsfw-detector",
    data={
        'image': 'https://cdn-pics.pornhd.com/pornhd/header/channel/314_phd_channel_big_Tushy-porn.jpg',
    },
    headers={'api-key': 'ba22a8a2-f054-491f-9604-d6a37760a72f'}
)
print(r.json())

# Example posting a local image file:
'''
import requests
r = requests.post(
    "https://api.deepai.org/api/nsfw-detector",
    files={
        'image': open('/path/to/your/file.jpg', 'rb'),
    },
    headers={'api-key': 'YOUR_API_KEY'}
)
print(r.json())
'''

{'output': {'detections': [{'confidence': '0.82', 'bounding_box': [560, 42, 74, 118], 'name': 'Male Genitalia - Exposed'}], 'nsfw_score': 0.994}}


'\nimport requests\nr = requests.post(\n    "https://api.deepai.org/api/nsfw-detector",\n    files={\n        \'image\': open(\'/path/to/your/file.jpg\', \'rb\'),\n    },\n    headers={\'api-key\': \'YOUR_API_KEY\'}\n)\nprint(r.json())\n'